# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(["punkt", "wordnet", "stopwords"])


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
import pickle
import re
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
import sqlalchemy as sa
from sqlalchemy import inspect

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = sa.create_engine("sqlite:///DisasterResponse.db")
inspector = inspect(engine)

for table_name in inspector.get_table_names():
    print("Table: %s" % table_name)

Table: Message


In [3]:
df = pd.read_sql_table("Message", engine)
df.head(2)

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
df.shape

(26216, 41)

In [5]:
df[df.columns[4:]].describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
for i in df[df.columns[4:]]:
    print(df[i].value_counts())

news      13054
direct    10766
social     2396
Name: genre, dtype: int64
1    19906
0     6122
2      188
Name: related, dtype: int64
0    21742
1     4474
Name: request, dtype: int64
0    26098
1      118
Name: offer, dtype: int64
0    15356
1    10860
Name: aid_related, dtype: int64
0    24132
1     2084
Name: medical_help, dtype: int64
0    24903
1     1313
Name: medical_products, dtype: int64
0    25492
1      724
Name: search_and_rescue, dtype: int64
0    25745
1      471
Name: security, dtype: int64
0    25356
1      860
Name: military, dtype: int64
0    26216
Name: child_alone, dtype: int64
0    24544
1     1672
Name: water, dtype: int64
0    23293
1     2923
Name: food, dtype: int64
0    23902
1     2314
Name: shelter, dtype: int64
0    25811
1      405
Name: clothing, dtype: int64
0    25612
1      604
Name: money, dtype: int64
0    25918
1      298
Name: missing_people, dtype: int64
0    25341
1      875
Name: refugees, dtype: int64
0    25022
1     1194
Name: death, dtype: 

In [7]:
# Remove child alone factor as it has only zeros
df = df.drop(["child_alone"],axis=1)

In [8]:
# There is a small portion of response type 2 in related column as I don't know if it's an error or not I decide to drop these rows
df = df[df.related !=2]

In [9]:
X = df.message
Y = df[df.columns[5:]]
cat_names = Y.columns

### 2. Write a tokenization function to process your text data

In [10]:
X.head(5)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [11]:
def tokenize(text: str) -> list:
    """
    Split text string into words and return list of words roots
    """
    # Normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenization
    words = word_tokenize(text)
    stop_words = stopwords.words("english")
    words = [word for word in words if word not in stop_words]
    
    # Lemmatization
    tokens = [WordNetLemmatizer().lemmatize(word) for word in words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
print(classification_report(Y_test.values, y_pred, target_names=Y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5016
               request       0.79      0.44      0.56      1130
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.61      0.67      2748
          medical_help       0.49      0.07      0.13       532
      medical_products       0.64      0.08      0.14       351
     search_and_rescue       0.50      0.06      0.11       183
              security       0.20      0.01      0.02       114
              military       0.81      0.10      0.17       229
                 water       0.82      0.28      0.42       432
                  food       0.85      0.46      0.59       743
               shelter       0.85      0.26      0.40       603
              clothing       0.95      0.17      0.29       109
                 money       0.75      0.04      0.08       136
        missing_people       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [17]:
parameters = {
    'clf__estimator__n_estimators': [10, 30],
    'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid = parameters, scoring='f1_micro')
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 30], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_pred_cv = cv.predict(X_test)

In [19]:
print(classification_report(Y_test.values, y_pred_cv, target_names=Y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      5016
               request       0.82      0.52      0.64      1130
                 offer       0.00      0.00      0.00        24
           aid_related       0.75      0.70      0.73      2748
          medical_help       0.68      0.10      0.17       532
      medical_products       0.83      0.14      0.23       351
     search_and_rescue       0.79      0.06      0.11       183
              security       0.00      0.00      0.00       114
              military       0.92      0.10      0.19       229
                 water       0.86      0.39      0.54       432
                  food       0.85      0.64      0.73       743
               shelter       0.84      0.41      0.55       603
              clothing       0.83      0.09      0.17       109
                 money       0.60      0.02      0.04       136
        missing_people       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline_ada.fit(X_train, Y_train)
y_pred_ada = pipeline_ada.predict(X_test)
print(classification_report(Y_test.values, y_pred_ada, target_names=Y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      5016
               request       0.76      0.52      0.62      1130
                 offer       0.00      0.00      0.00        24
           aid_related       0.77      0.59      0.67      2748
          medical_help       0.55      0.26      0.35       532
      medical_products       0.66      0.35      0.46       351
     search_and_rescue       0.50      0.18      0.27       183
              security       0.40      0.09      0.14       114
              military       0.65      0.32      0.43       229
                 water       0.76      0.65      0.70       432
                  food       0.81      0.66      0.73       743
               shelter       0.76      0.54      0.64       603
              clothing       0.75      0.51      0.61       109
                 money       0.54      0.38      0.44       136
        missing_people       0.59      

In [21]:
pipeline_nb = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])

pipeline_nb.fit(X_train, Y_train)
y_pred_nb = pipeline_nb.predict(X_test)
print(classification_report(Y_test.values, y_pred_nb, target_names=Y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.79      0.99      0.88      5016
               request       0.88      0.23      0.36      1130
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.63      0.68      2748
          medical_help       0.67      0.00      0.01       532
      medical_products       0.00      0.00      0.00       351
     search_and_rescue       0.00      0.00      0.00       183
              security       0.00      0.00      0.00       114
              military       0.00      0.00      0.00       229
                 water       0.00      0.00      0.00       432
                  food       0.77      0.02      0.04       743
               shelter       0.00      0.00      0.00       603
              clothing       0.00      0.00      0.00       109
                 money       0.00      0.00      0.00       136
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
pipeline_ada.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [23]:
ada_params= {
    'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
    'clf__estimator__n_estimators': [10, 20, 40]
    }

In [24]:
cv_ada = GridSearchCV(pipeline_ada, param_grid = ada_params, scoring='f1_micro')
cv_ada.fit(X_train, Y_train)
y_pred_cvada = cv_ada.predict(X_test)
print(classification_report(Y_test.values, y_pred_cvada, target_names=Y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5016
               request       0.82      0.13      0.22      1130
                 offer       0.00      0.00      0.00        24
           aid_related       0.82      0.20      0.32      2748
          medical_help       0.61      0.10      0.17       532
      medical_products       0.50      0.00      0.01       351
     search_and_rescue       0.58      0.19      0.29       183
              security       0.00      0.00      0.00       114
              military       0.56      0.13      0.21       229
                 water       0.56      0.84      0.67       432
                  food       0.78      0.69      0.73       743
               shelter       0.86      0.31      0.45       603
              clothing       1.00      0.01      0.02       109
                 money       0.54      0.18      0.27       136
        missing_people       0.79      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [26]:
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(pipeline_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.